---

## 1. Učitavanje potrebnih biblioteka i učitavanje podataka

Prvo ćemo učitati sve potrebne biblioteke za analizu podataka:

In [65]:
# Učitavanje potrebnih biblioteka
import pandas as pd         
import numpy as np           

# Prikaži verzije biblioteka
print("Verzije biblioteka:")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# Učitavanje podataka iz CSV datoteke iz dataset mape
# read_csv je standardna Pandas funkcija za učitavanje podataka iz CSV formata
df_original = pd.read_csv('../dataset/student-por.csv', sep=';')

# Kreiramo radnu kopiju s kojom ćemo raditi (backup originala u memoriji)
df = df_original.copy()

print("Datoteka uspješno učitana!")
print(f"\nDimenzije datoteke: {df.shape}")
print(f"  - Broj redaka (zapisa): {df.shape[0]}")
print(f"  - Broj stupaca (atributa): {df.shape[1]}")

Verzije biblioteka:
Pandas: 2.3.3
NumPy: 2.4.0
Datoteka uspješno učitana!

Dimenzije datoteke: (649, 33)
  - Broj redaka (zapisa): 649
  - Broj stupaca (atributa): 33


### Prikaz Osnovnih Informacija o DataFrameu

**Što prikazujemo:**
- Nazive svih stupaca (atributa)
- Tipove podataka u svakom stupcu
- Broj ne-nedostajućih vrijednosti
- Memorijsku potrošnju

In [66]:
# Detaljne informacije o DataFrameu
print("INFORMACIJE O DATAFREMU - PRIJE OBRADE:\n")
df.info()

INFORMACIJE O DATAFREMU - PRIJE OBRADE:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery  

### Prikaz Prvih Redaka

Pogledajmo kako izgledaju prvi redci dataseta:

In [67]:
# Prikazi prvih 10 redaka
print("PRVIH 10 REDAKA DATASETA:\n")
print(df.head(10))
print("\n" + "="*60 + "\n")

PRVIH 10 REDAKA DATASETA:

  school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4   at_home   teacher   
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   15       U     LE3       T     1     1   at_home     other   
3     GP   F   15       U     GT3       T     4     2    health  services   
4     GP   F   16       U     GT3       T     3     3     other     other   
5     GP   M   16       U     LE3       T     4     3  services     other   
6     GP   M   16       U     LE3       T     2     2     other     other   
7     GP   F   17       U     GT3       A     4     4     other   teacher   
8     GP   M   15       U     LE3       A     3     2  services     other   
9     GP   M   15       U     GT3       T     3     4     other     other   

   ... famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0  ...      4        3      4     1     1      3     

---

## 2. Otkrivanje Nedostajućih Vrijednosti

Sada ćemo analizirati gdje su nedostajuće vrijednosti u našem datasetu. Pandas koristi `NaN` (Not a Number) za označavanje nedostajućih vrijednosti.


### Prikaz redaka s nedostajućim vrijednostima

`.isnull().any(axis=1)` provjerava je li bilo koja vrijednost u redu NaN (`any(axis=1)` znači "provjeravaj redove" - axis=1), što nam daje sve redake koji imaju barem jednu nedostajuću vrijednost.

In [68]:
# Pronađi sve redove koji imaju BAR JEDNU nedostajuću vrijednost
# .any(axis=1) provjerava po redcima (axis=1) - vraća True ako red ima bar jednu NaN vrijednost
rows_with_missing = df[df.isnull().any(axis=1)]

print(f"REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA ({len(rows_with_missing)} redaka):\n")
print(rows_with_missing)

REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA (0 redaka):

Empty DataFrame
Columns: [school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G2, G3]
Index: []

[0 rows x 33 columns]


---

## 3. Rekodiranje Nedostajućih Vrijednosti

Sada ćemo **zamijeniti sve nedostajuće vrijednosti (NaN)** sa stringom **"missing"**.

### Zašto rekodirati nedostajuće vrijednosti?

1. **Jasnoća** - lakše je razumjeti podatke kada je eksplicitno navedeno što je "nedostaje"
2. **Analiza** - "missing" je stringovna vrijednost koja se može analizirati kao kategorija
3. **Usporedba** - lakše se može pregledati koji stupac ima više nedostajućih vrijednosti
4. **Čuvanje podataka** - čuva se informacija da je vrijednost nedostajala, za razliku od brisanja redaka

### Metoda: fillna() - Zamjena nedostajućih vrijednosti

`.fillna('missing')` zamjenjuje sve NaN vrijednosti navedenom vrijednosti (u ovom slučaju stringom "missing"), što omogućava da nedostajuće vrijednosti postanu vidljive kao eksplicitne kategorije umjesto numeričkog NaN.

In [69]:
# Zamjena svih NaN vrijednosti sa stringom "missing"
# fillna() metoda zamjenjuje sve NaN vrijednosti navedenom vrijednosti

print("REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:\n")
print("Korištenje .fillna('missing')...\n")

df_cleaned = df.fillna('missing')

print("Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.")
print("\n" + "="*60 + "\n")

REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:

Korištenje .fillna('missing')...

Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.




### Provjera rezultata rekodiranja

Pogledajmo redake s nedostajućim vrijednostima nakon rekodiranja:

In [71]:
# Prikaži redake koji su prije imali nedostajuće vrijednosti
print("REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):\n")
print(rows_with_missing.index.tolist())
print("\nPrikaz tih redaka iz očišćenog dataseta:\n")
print(df_cleaned.loc[rows_with_missing.index])
print("\n" + "="*60 + "\n")

REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):

[]

Prikaz tih redaka iz očišćenog dataseta:

Empty DataFrame
Columns: [school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G2, G3]
Index: []

[0 rows x 33 columns]


